In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import hypergeom, mannwhitneyu
import matplotlib.patches as mpatches
import re
from statsmodels.stats.multitest import fdrcorrection
import matplotlib as mpl
import warnings
import gseapy as gp
from gseapy import enrichr, gsea, prerank
import os
import textwrap

# Suppress warnings
warnings.filterwarnings("ignore")

# Set plot style
plt.style.use('default')
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial']
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.alpha'] = 0.3
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = 0.8

print("=" * 80)
print("COMPLETE GSEA ANALYSIS FOR NANOSTRING AND OLINK DATA")
print("Using Wilcoxon Rank-Sum Tests for Differential Expression")
print("All Three Comparisons: EXT vs NON, EXT vs INT, INT vs NON")
print("=" * 80)

# Create output directories
os.makedirs('gsea_results_complete', exist_ok=True)
os.makedirs('gsea_results_complete/nanostring', exist_ok=True)
os.makedirs('gsea_results_complete/olink', exist_ok=True)

# Define comparisons
comparisons = [
    ('EXT', 'NON', 'ext_vs_non'),
    ('EXT', 'INT', 'ext_vs_int'), 
    ('INT', 'NON', 'int_vs_non')
]

# ============================================================================
# NANOSTRING DATA ANALYSIS AND GSEA
# ============================================================================

print("\n" + "="*50)
print("NANOSTRING DATA ANALYSIS")
print("="*50)

print("Loading Nanostring data...")
# Load the Nanostring data
nanostring_df = pd.read_csv('nanostring_data.csv')

# Load the sample info
sample_info_df = pd.read_csv('sample_info.csv', quotechar='"')

# Clean up column names and values
sample_info_df.columns = sample_info_df.columns.str.strip('"')
for col in sample_info_df.columns:
    if sample_info_df[col].dtype == 'object':
        sample_info_df[col] = sample_info_df[col].str.strip('"')

# Define valid response categories
valid_responses = {'EXT', 'INT', 'NON'}

# Create mappings from sample IDs to response groups and patient IDs
sample_to_response = {}
sample_to_patientid = {}
sample_to_timepoint = {}
sample_columns = nanostring_df.columns[3:]  # Skip first 3 columns

# Parse timepoint from sample ID
def extract_timepoint(sample_id):
    match = re.search(r'\.([^.]+)$', sample_id)
    if match:
        return match.group(1)
    return None

for _, row in sample_info_df.iterrows():
    if pd.notna(row['Sample #s']):
        samples = row['Sample #s'].split(',')
        patient_id = row['ID']
        response = row['Response (Extreme, Intermediate, Non)']
        
        # Only map samples with valid response categories
        if response in valid_responses:
            for sample in samples:
                sample = sample.strip()
                if sample in sample_columns:
                    sample_to_response[sample] = response
                    sample_to_patientid[sample] = patient_id
                    
                    timepoint = extract_timepoint(sample)
                    sample_to_timepoint[sample] = timepoint

print(f"Nanostring: Samples mapped to valid response groups: {len(sample_to_response)}")

# Count samples by response group
response_counts = {}
for response in valid_responses:
    response_counts[response] = sum(1 for r in sample_to_response.values() if r == response)
print(f"Sample counts - EXT: {response_counts.get('EXT', 0)}, INT: {response_counts.get('INT', 0)}, NON: {response_counts.get('NON', 0)}")

# Reshape data for analysis
marker_data = nanostring_df.set_index('Marker Name')
marker_data = marker_data.drop(['Accession', 'Code.Class'], axis=1)

# Function to compare expression levels between response groups using Wilcoxon
def compare_groups_nano(data, marker, group1, group2, sample_groups):
    """Compare expression levels of a marker between two groups using Wilcoxon rank-sum test"""
    group1_cols = [col for col in data.columns if sample_groups.get(col) == group1]
    group2_cols = [col for col in data.columns if sample_groups.get(col) == group2]
    
    if len(group1_cols) > 0 and len(group2_cols) > 0:
        group1_vals = data.loc[marker, group1_cols].astype(float)
        group2_vals = data.loc[marker, group2_cols].astype(float)
        
        # Remove NaN values
        group1_vals = group1_vals.dropna()
        group2_vals = group2_vals.dropna()
        
        if len(group1_vals) < 2 or len(group2_vals) < 2:
            return None
        
        mean_diff = group1_vals.mean() - group2_vals.mean()
        
        # Calculate log2 fold change
        eps = 1e-10
        log2_fold_change = np.log2((group1_vals.mean() + eps) / (group2_vals.mean() + eps))
        
        # Perform Wilcoxon rank-sum test (Mann-Whitney U test)
        try:
            statistic, p_val = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
        except ValueError:
            # Handle case where all values are identical
            p_val = 1.0
            statistic = 0
        
        # Effect size (rank-biserial correlation) - similar to Cohen's d for non-parametric
        n1, n2 = len(group1_vals), len(group2_vals)
        effect_size = 2 * statistic / (n1 * n2) - 1  # Rank-biserial correlation
        
        return {
            'p_value': p_val,
            'fold_change': mean_diff,
            'log2_fold_change': log2_fold_change,
            'mean_group1': group1_vals.mean(),
            'mean_group2': group2_vals.mean(),
            'median_group1': group1_vals.median(),
            'median_group2': group2_vals.median(),
            'std_group1': group1_vals.std(),
            'std_group2': group2_vals.std(),
            'n_group1': len(group1_vals),
            'n_group2': len(group2_vals),
            'wilcoxon_statistic': statistic,
            'effect_size': effect_size
        }
    return None

# Perform all three comparisons for Nanostring
nano_comparison_results = {}

for group1, group2, comp_name in comparisons:
    print(f"Nanostring: Comparing {group1} vs {group2} using Wilcoxon rank-sum test...")
    results = {}
    for marker in marker_data.index:
        result = compare_groups_nano(marker_data, marker, group1, group2, sample_to_response)
        if result:
            results[marker] = result
    nano_comparison_results[comp_name] = results
    print(f"  Analyzed {len(results)} markers for {group1} vs {group2}")

# Define gene sets to use
gene_sets = [
    'GO_Biological_Process_2021',
    'GO_Molecular_Function_2021', 
    'GO_Cellular_Component_2021',
    'KEGG_2021_Human',
    'WikiPathway_2021_Human',
    'Reactome_2022'
]

# Run GSEA for all Nanostring comparisons
nano_gsea_results = {}
nano_gsea_summaries = {}

for group1, group2, comp_name in comparisons:
    print(f"\nRunning Nanostring GSEA for {group1} vs {group2}...")
    
    # Create output directory for this comparison
    os.makedirs(f'gsea_results_complete/nanostring/{comp_name}', exist_ok=True)
    
    # Prepare ranked gene list
    results = nano_comparison_results[comp_name]
    ranked_genes = [(gene, result['log2_fold_change']) for gene, result in results.items()]
    ranked_genes.sort(key=lambda x: x[1], reverse=True)
    
    # Create ranked gene list for GSEA
    rnk = pd.DataFrame(ranked_genes, columns=['gene', 'score'])
    rnk.to_csv(f'gsea_results_complete/nanostring/{comp_name}_ranked.rnk', sep='\t', index=False, header=False)
    
    # Run GSEA for each gene set
    comp_gsea_results = {}
    for gene_set in gene_sets:
        print(f"  Running GSEA for {gene_set}...")
        try:
            pre_res = gp.prerank(
                rnk=rnk,
                gene_sets=gene_set,
                processes=1,
                permutation_num=1000,
                outdir=f'gsea_results_complete/nanostring/{comp_name}/{gene_set}',
                format='png',
                seed=42,
                min_size=15,
                max_size=500
            )
            comp_gsea_results[gene_set] = pre_res
            print(f"    Completed {gene_set}: Found {len(pre_res.res2d)} enriched pathways")
        except Exception as e:
            print(f"    Error with {gene_set}: {str(e)}")
    
    nano_gsea_results[comp_name] = comp_gsea_results
    
    # Create summary for this comparison
    gsea_summary = []
    for gene_set, results in comp_gsea_results.items():
        if results is not None and hasattr(results, 'res2d'):
            for _, row in results.res2d.iterrows():
                gsea_summary.append({
                    'Comparison': f'{group1}_vs_{group2}',
                    'Gene_Set_Database': gene_set,
                    'Term': row['Term'],
                    'ES': row['ES'],
                    'NES': row['NES'],
                    'NOM p-val': row['NOM p-val'],
                    'FDR q-val': row['FDR q-val'],
                    'FWER p-val': row['FWER p-val'],
                    'Tag %': row['Tag %'],
                    'Gene %': row['Gene %'],
                    'Lead_genes': row['Lead_genes']
                })
    
    if gsea_summary:
        summary_df = pd.DataFrame(gsea_summary)
        summary_df = summary_df.sort_values(['FDR q-val', 'NOM p-val'])
        summary_df.to_csv(f'gsea_results_complete/nanostring/nanostring_{comp_name}_gsea_summary.csv', index=False)
        nano_gsea_summaries[comp_name] = summary_df
        print(f"  Saved GSEA summary with {len(summary_df)} enriched pathways")

# ============================================================================
# OLINK DATA ANALYSIS AND GSEA
# ============================================================================

print("\n" + "="*50)
print("OLINK DATA ANALYSIS")
print("="*50)

print("Loading Olink protein data...")
# Read the Olink data
olink_raw = pd.read_csv('olink_data.csv')

# Extract sample IDs (rows 3 onwards, first column)
sample_ids = olink_raw.iloc[3:, 0].tolist()

# Extract the protein columns
panel_row = olink_raw.iloc[0, 1:].tolist()

# Create protein labels
protein_cols = []
protein_counts = {}

for panel in panel_row:
    if pd.isna(panel) or 'Plate ID' in str(panel) or 'QC Warning' in str(panel):
        protein_cols.append(None)
    else:
        if isinstance(panel, str):
            parts = panel.split('(')
            if len(parts) > 0:
                protein_name = parts[0].strip()
            else:
                protein_name = panel
        else:
            protein_name = str(panel)
        
        if protein_name in protein_counts:
            protein_counts[protein_name] += 1
            unique_name = f"{protein_name}_{protein_counts[protein_name]}"
        else:
            protein_counts[protein_name] = 1
            unique_name = protein_name
        
        protein_cols.append(unique_name)

# Create protein matrix
protein_matrix = pd.DataFrame(index=[p for p in protein_cols if p is not None], columns=sample_ids)

# Fill the matrix with expression values
for idx, sample_id in enumerate(sample_ids):
    data_row = olink_raw.iloc[idx + 3, 1:]
    
    for val, protein_name in zip(data_row, protein_cols):
        if protein_name is not None:
            protein_matrix.loc[protein_name, sample_id] = pd.to_numeric(val, errors='coerce')

print(f"Olink: Processed protein matrix shape: {protein_matrix.shape}")

# Map Olink samples to response groups
olink_sample_to_response = {}
olink_sample_to_patientid = {}

def extract_olink_timepoint(sample_id):
    match = re.search(r'([B]\d+[W]\d+)', sample_id)
    if match:
        return match.group(1)
    return None

for _, row in sample_info_df.iterrows():
    if pd.notna(row['Response (Extreme, Intermediate, Non)']) and row['Response (Extreme, Intermediate, Non)'] in valid_responses:
        patient_id = row['ID']
        if pd.notna(row['Timepoints collected']):
            info_sample_ids = row['Timepoints collected'].split(',')
            for info_sample_id in info_sample_ids:
                info_sample_id = info_sample_id.strip()
                if info_sample_id in protein_matrix.columns:
                    olink_sample_to_response[info_sample_id] = row['Response (Extreme, Intermediate, Non)']
                    olink_sample_to_patientid[info_sample_id] = patient_id

print(f"Olink: Mapped {len(olink_sample_to_response)} samples to valid response groups")

# Count Olink samples by response group
olink_response_counts = {}
for response in valid_responses:
    olink_response_counts[response] = sum(1 for r in olink_sample_to_response.values() if r == response)
print(f"Olink sample counts - EXT: {olink_response_counts.get('EXT', 0)}, INT: {olink_response_counts.get('INT', 0)}, NON: {olink_response_counts.get('NON', 0)}")

# Function to compare protein expression between groups using Wilcoxon
def compare_groups_olink(data, protein, group1, group2, sample_groups):
    """Compare expression levels of a protein between two groups using Wilcoxon rank-sum test"""
    group1_samples = [s for s in data.columns if s in sample_groups and sample_groups[s] == group1]
    group2_samples = [s for s in data.columns if s in sample_groups and sample_groups[s] == group2]
    
    if len(group1_samples) > 0 and len(group2_samples) > 0:
        try:
            group1_vals = data.loc[protein, group1_samples].astype(float)
            group2_vals = data.loc[protein, group2_samples].astype(float)
            
            group1_vals = group1_vals.dropna()
            group2_vals = group2_vals.dropna()
            
            if len(group1_vals) < 2 or len(group2_vals) < 2:
                return None
            
            mean_diff = group1_vals.mean() - group2_vals.mean()
            
            eps = 1e-10
            log2_fold_change = np.log2((group1_vals.mean() + eps) / (group2_vals.mean() + eps))
            
            # Perform Wilcoxon rank-sum test (Mann-Whitney U test)
            try:
                statistic, p_val = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
            except ValueError:
                # Handle case where all values are identical
                p_val = 1.0
                statistic = 0
            
            # Effect size (rank-biserial correlation)
            n1, n2 = len(group1_vals), len(group2_vals)
            effect_size = 2 * statistic / (n1 * n2) - 1  # Rank-biserial correlation
            
            return {
                'p_value': p_val,
                'fold_change': mean_diff,
                'log2_fold_change': log2_fold_change,
                'mean_group1': group1_vals.mean(),
                'mean_group2': group2_vals.mean(),
                'median_group1': group1_vals.median(),
                'median_group2': group2_vals.median(),
                'std_group1': group1_vals.std(),
                'std_group2': group2_vals.std(),
                'n_group1': len(group1_vals),
                'n_group2': len(group2_vals),
                'wilcoxon_statistic': statistic,
                'effect_size': effect_size
            }
        except Exception as e:
            return None
    return None

# Perform all three comparisons for Olink
olink_comparison_results = {}

for group1, group2, comp_name in comparisons:
    print(f"Olink: Comparing {group1} vs {group2} using Wilcoxon rank-sum test...")
    results = {}
    for protein in protein_matrix.index:
        result = compare_groups_olink(protein_matrix, protein, group1, group2, olink_sample_to_response)
        if result:
            results[protein] = result
    olink_comparison_results[comp_name] = results
    print(f"  Analyzed {len(results)} proteins for {group1} vs {group2}")

# For protein data, we'll use protein-centric gene sets
protein_gene_sets = [
    'GO_Biological_Process_2021',
    'GO_Molecular_Function_2021',
    'KEGG_2021_Human',
    'Reactome_2022',
    'WikiPathway_2021_Human'
]

# Run GSEA for all Olink comparisons
olink_gsea_results = {}
olink_gsea_summaries = {}

for group1, group2, comp_name in comparisons:
    print(f"\nRunning Olink GSEA for {group1} vs {group2}...")
    
    # Create output directory for this comparison
    os.makedirs(f'gsea_results_complete/olink/{comp_name}', exist_ok=True)
    
    # Prepare ranked protein list
    results = olink_comparison_results[comp_name]
    ranked_proteins = [(protein, result['log2_fold_change']) for protein, result in results.items()]
    ranked_proteins.sort(key=lambda x: x[1], reverse=True)
    
    # Create ranked protein list for GSEA
    rnk = pd.DataFrame(ranked_proteins, columns=['protein', 'score'])
    rnk.to_csv(f'gsea_results_complete/olink/{comp_name}_ranked.rnk', sep='\t', index=False, header=False)
    
    # Run GSEA for each gene set
    comp_gsea_results = {}
    for gene_set in protein_gene_sets:
        print(f"  Running GSEA for {gene_set}...")
        try:
            pre_res = gp.prerank(
                rnk=rnk,
                gene_sets=gene_set,
                processes=1,
                permutation_num=1000,
                outdir=f'gsea_results_complete/olink/{comp_name}/{gene_set}',
                format='png',
                seed=42,
                min_size=5,  # Lower min_size for proteins
                max_size=500
            )
            comp_gsea_results[gene_set] = pre_res
            print(f"    Completed {gene_set}: Found {len(pre_res.res2d)} enriched pathways")
        except Exception as e:
            print(f"    Error with {gene_set}: {str(e)}")
    
    olink_gsea_results[comp_name] = comp_gsea_results
    
    # Create summary for this comparison
    gsea_summary = []
    for gene_set, results in comp_gsea_results.items():
        if results is not None and hasattr(results, 'res2d'):
            for _, row in results.res2d.iterrows():
                gsea_summary.append({
                    'Comparison': f'{group1}_vs_{group2}',
                    'Gene_Set_Database': gene_set,
                    'Term': row['Term'],
                    'ES': row['ES'],
                    'NES': row['NES'],
                    'NOM p-val': row['NOM p-val'],
                    'FDR q-val': row['FDR q-val'],
                    'FWER p-val': row['FWER p-val'],
                    'Tag %': row['Tag %'],
                    'Gene %': row['Gene %'],
                    'Lead_genes': row['Lead_genes']
                })
    
    if gsea_summary:
        summary_df = pd.DataFrame(gsea_summary)
        summary_df = summary_df.sort_values(['FDR q-val', 'NOM p-val'])
        summary_df.to_csv(f'gsea_results_complete/olink/olink_{comp_name}_gsea_summary.csv', index=False)
        olink_gsea_summaries[comp_name] = summary_df
        print(f"  Saved GSEA summary with {len(summary_df)} enriched pathways")

# ============================================================================
# COMPREHENSIVE VISUALIZATION AND COMPARISON
# ============================================================================

print("\n" + "="*50)
print("CREATING COMPREHENSIVE VISUALIZATIONS")
print("="*50)

def create_comparison_summary_plot(summaries_dict, data_type, output_path):
    """Create comprehensive summary plot comparing all three comparisons"""
    
    print(f"Creating summary plot for {data_type}...")
    print(f"Available comparisons: {list(summaries_dict.keys())}")
    
    # Combine all summaries
    all_data = []
    for comp_name, summary_df in summaries_dict.items():
        if summary_df is not None and len(summary_df) > 0:
            print(f"  {comp_name}: {len(summary_df)} total pathways")
            # Get top 10 significant pathways for each comparison
            sig_df = summary_df[summary_df['FDR q-val'] < 0.25].head(10)
            print(f"  {comp_name}: {len(sig_df)} significant pathways (FDR < 0.25)")
            if len(sig_df) > 0:
                sig_df = sig_df.copy()
                sig_df['Comparison'] = comp_name.replace('_', ' ').upper()
                all_data.append(sig_df)
    
    if not all_data:
        print(f"No significant pathways found for {data_type}")
        # Create empty plot with message
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        ax.text(0.5, 0.5, f'No significant pathways found\nfor {data_type}\n(FDR < 0.25)', 
               ha='center', va='center', transform=ax.transAxes, fontsize=16)
        ax.set_title(f'{data_type} GSEA Results\n(Wilcoxon Rank-Sum Tests)', fontsize=18, fontweight='bold')
        ax.axis('off')
        plt.tight_layout()
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        return
    
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"Combined dataframe shape: {combined_df.shape}")
    
    # Calculate dynamic figure size based on number of pathways
    max_pathways_per_plot = max([len(combined_df[combined_df['Comparison'] == comp]) 
                                for comp in combined_df['Comparison'].unique()])
    
    # Create the plot with better spacing
    fig_width = 30  # Increased width
    fig_height = max(8, max_pathways_per_plot * 0.8 + 4)  # Dynamic height
    
    fig, axes = plt.subplots(1, 3, figsize=(fig_width, fig_height))
    
    comparison_names = ['EXT VS NON', 'EXT VS INT', 'INT VS NON']
    colors = ['#e41a1c', '#377eb8', '#4daf4a']
    
    for i, (comp_name, color) in enumerate(zip(comparison_names, colors)):
        ax = axes[i]
        comp_data = combined_df[combined_df['Comparison'] == comp_name]
        
        print(f"  Plotting {comp_name}: {len(comp_data)} pathways")
        
        if len(comp_data) > 0:
            # Color by NES (positive = red, negative = blue)
            bar_colors = ['darkred' if nes > 0 else 'darkblue' for nes in comp_data['NES']]
            
            y_pos = np.arange(len(comp_data))
            bars = ax.barh(y_pos, -np.log10(comp_data['FDR q-val']), color=bar_colors, alpha=0.7)
            
            # Customize each subplot - better text handling
            ax.set_yticks(y_pos)
            
            # Truncate long pathway names and wrap intelligently
            wrapped_labels = []
            for term in comp_data['Term']:
                term_str = str(term)
                # First truncate if extremely long
                if len(term_str) > 60:
                    term_str = term_str[:57] + "..."
                
                # Then wrap for display with shorter width
                if len(term_str) > 25:
                    wrapped = '\n'.join(textwrap.wrap(term_str, width=25))
                    # Limit to max 3 lines to prevent excessive height
                    lines = wrapped.split('\n')
                    if len(lines) > 3:
                        wrapped = '\n'.join(lines[:2]) + '\n' + lines[2][:22] + '...'
                else:
                    wrapped = term_str
                wrapped_labels.append(wrapped)
            
            ax.set_yticklabels(wrapped_labels, fontsize=7, ha='right')
            ax.set_xlabel('-Log10(FDR q-value)', fontsize=12)
            ax.set_title(f'{comp_name}\n{data_type}', fontsize=14, fontweight='bold', pad=20)
            ax.axvline(-np.log10(0.25), color='gray', linestyle='--', alpha=0.5, label='FDR = 0.25')
            ax.grid(True, alpha=0.3)
            
            # Adjust margins and limits
            ax.margins(y=0.02)
            
            # Ensure x-axis starts from 0
            ax.set_xlim(left=0)
            
        else:
            ax.text(0.5, 0.5, 'No significant\npathways found', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=12)
            ax.set_title(f'{comp_name}\n{data_type}', fontsize=14, fontweight='bold', pad=20)
            ax.set_xlabel('-Log10(FDR q-value)', fontsize=12)
    
    # Add overall legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='darkred', alpha=0.7, label='Positive NES (Upregulated)'),
        Patch(facecolor='darkblue', alpha=0.7, label='Negative NES (Downregulated)'),
        plt.Line2D([0], [0], color='gray', linestyle='--', alpha=0.5, label='FDR = 0.25')
    ]
    fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize=10)
    
    # Adjust layout with much more space for y-axis labels
    plt.tight_layout()
    plt.subplots_adjust(left=0.35, bottom=0.12, right=0.98, top=0.92, wspace=0.4)
    
    # Save with high DPI and tight bounding box
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"Saved comprehensive comparison plot: {output_path}")

# Create comprehensive summary plots
print("Checking available GSEA results...")
print(f"Nanostring summaries available: {list(nano_gsea_summaries.keys()) if 'nano_gsea_summaries' in locals() else 'None'}")
print(f"Olink summaries available: {list(olink_gsea_summaries.keys()) if 'olink_gsea_summaries' in locals() else 'None'}")

if 'nano_gsea_summaries' in locals() and nano_gsea_summaries:
    print("Creating Nanostring summary plot...")
    create_comparison_summary_plot(nano_gsea_summaries, 'Nanostring', 
                                 'gsea_results_complete/nanostring_all_comparisons_summary.png')
else:
    print("No Nanostring GSEA summaries found")

if 'olink_gsea_summaries' in locals() and olink_gsea_summaries:
    print("Creating Olink summary plot...")
    create_comparison_summary_plot(olink_gsea_summaries, 'Olink', 
                                 'gsea_results_complete/olink_all_comparisons_summary.png')
else:
    print("No Olink GSEA summaries found")

# Create cross-platform comparison
def create_cross_platform_comparison():
    """Compare pathways across platforms and comparisons"""
    
    print("Creating cross-platform pathway comparison...")
    
    all_pathways = []
    
    # Collect all significant pathways from both platforms
    for comp_name in ['ext_vs_non', 'ext_vs_int', 'int_vs_non']:
        comp_display = comp_name.replace('_', ' ').upper()
        
        # Nanostring pathways
        if 'nano_gsea_summaries' in locals() and comp_name in nano_gsea_summaries:
            nano_df = nano_gsea_summaries[comp_name]
            nano_sig = nano_df[nano_df['FDR q-val'] < 0.25]
            print(f"  {comp_display} Nanostring: {len(nano_sig)} significant pathways")
            for _, row in nano_sig.iterrows():
                all_pathways.append({
                    'Platform': 'Nanostring', 
                    'Comparison': comp_display,
                    'Pathway': row['Term'][:60],
                    'FDR': row['FDR q-val'],
                    'NES': row['NES']
                })
        
        # Olink pathways  
        if 'olink_gsea_summaries' in locals() and comp_name in olink_gsea_summaries:
            olink_df = olink_gsea_summaries[comp_name]
            olink_sig = olink_df[olink_df['FDR q-val'] < 0.25]
            print(f"  {comp_display} Olink: {len(olink_sig)} significant pathways")
            for _, row in olink_sig.iterrows():
                all_pathways.append({
                    'Platform': 'Olink',
                    'Comparison': comp_display, 
                    'Pathway': row['Term'][:60],
                    'FDR': row['FDR q-val'],
                    'NES': row['NES']
                })
    
    if all_pathways:
        pathway_df = pd.DataFrame(all_pathways)
        pathway_df.to_csv('gsea_results_complete/cross_platform_pathway_comparison.csv', index=False)
        print(f"Saved cross-platform comparison with {len(all_pathways)} total pathways")
        
        # Find overlapping pathways
        nano_pathways = set(pathway_df[pathway_df['Platform'] == 'Nanostring']['Pathway'])
        olink_pathways = set(pathway_df[pathway_df['Platform'] == 'Olink']['Pathway'])
        overlapping = nano_pathways.intersection(olink_pathways)
        
        print(f"Cross-platform analysis:")
        print(f"  Nanostring significant pathways: {len(nano_pathways)}")
        print(f"  Olink significant pathways: {len(olink_pathways)}")
        print(f"  Overlapping pathways: {len(overlapping)}")
        
        if overlapping:
            print("  Overlapping pathways:")
            for pathway in sorted(overlapping):
                print(f"    - {pathway}")
    else:
        print("No significant pathways found for cross-platform comparison")

# Run cross-platform comparison
create_cross_platform_comparison()

# Save differential expression results for all comparisons
print("\nSaving comprehensive differential expression results...")

for group1, group2, comp_name in comparisons:
    # Nanostring differential expression
    if comp_name in nano_comparison_results:
        nano_de_results = []
        for gene, result in nano_comparison_results[comp_name].items():
            nano_de_results.append({
                'Gene': gene,
                'Log2_Fold_Change': result['log2_fold_change'],
                'P_Value': result['p_value'],
                'Wilcoxon_Statistic': result['wilcoxon_statistic'],
                'Effect_Size': result['effect_size'],
                f'Mean_{group1}': result['mean_group1'],
                f'Mean_{group2}': result['mean_group2'],
                f'Median_{group1}': result['median_group1'],
                f'Median_{group2}': result['median_group2']
            })
        
        nano_de_df = pd.DataFrame(nano_de_results)
        nano_de_df = nano_de_df.sort_values('P_Value')
        nano_de_df.to_csv(f'gsea_results_complete/nanostring_{comp_name}_differential_expression_wilcoxon.csv', index=False)
    
    # Olink differential expression
    if comp_name in olink_comparison_results:
        olink_de_results = []
        for protein, result in olink_comparison_results[comp_name].items():
            olink_de_results.append({
                'Protein': protein,
                'Log2_Fold_Change': result['log2_fold_change'],
                'P_Value': result['p_value'],
                'Wilcoxon_Statistic': result['wilcoxon_statistic'],
                'Effect_Size': result['effect_size'],
                f'Mean_{group1}': result['mean_group1'],
                f'Mean_{group2}': result['mean_group2'],
                f'Median_{group1}': result['median_group1'],
                f'Median_{group2}': result['median_group2']
            })
        
        olink_de_df = pd.DataFrame(olink_de_results)
        olink_de_df = olink_de_df.sort_values('P_Value')
        olink_de_df.to_csv(f'gsea_results_complete/olink_{comp_name}_differential_expression_wilcoxon.csv', index=False)

print("\n" + "="*80)
print("COMPLETE GSEA ANALYSIS FINISHED - WILCOXON VERSION")
print("="*80)

# Summary statistics
total_nano_genes = sum(len(results) for results in nano_comparison_results.values())
total_olink_proteins = sum(len(results) for results in olink_comparison_results.values())
total_nano_pathways = sum(len(summary) for summary in nano_gsea_summaries.values() if summary is not None)
total_olink_pathways = sum(len(summary) for summary in olink_gsea_summaries.values() if summary is not None)

print(f"Analysis Summary (Using Wilcoxon Rank-Sum Tests):")
print(f"  Nanostring: {total_nano_genes} total gene comparisons across 3 contrasts")
print(f"  Olink: {total_olink_proteins} total protein comparisons across 3 contrasts")
print(f"  Nanostring GSEA: {total_nano_pathways} total enriched pathways")
print(f"  Olink GSEA: {total_olink_pathways} total enriched pathways")

print(f"\nKey output files in 'gsea_results_complete/' directory:")
print(f"  - Individual comparison results in nanostring/ and olink/ subdirectories")
print(f"  - *_gsea_summary.csv files for each comparison")
print(f"  - *_differential_expression_wilcoxon.csv files for each comparison") 
print(f"  - nanostring_all_comparisons_summary.png")
print(f"  - olink_all_comparisons_summary.png")
print(f"  - cross_platform_pathway_comparison.csv")

print(f"\nStatistical Method Changes:")
print(f"  - Replaced t-tests with Wilcoxon rank-sum tests (Mann-Whitney U)")
print(f"  - Added effect size calculations (rank-biserial correlation)")
print(f"  - Added median values alongside means")
print(f"  - Maintained log2 fold change calculations for GSEA ranking")
print(f"  - All p-values now from non-parametric tests")

COMPLETE GSEA ANALYSIS FOR NANOSTRING AND OLINK DATA
Using Wilcoxon Rank-Sum Tests for Differential Expression
All Three Comparisons: EXT vs NON, EXT vs INT, INT vs NON

NANOSTRING DATA ANALYSIS
Loading Nanostring data...
Nanostring: Samples mapped to valid response groups: 58
Sample counts - EXT: 28, INT: 22, NON: 8
Nanostring: Comparing EXT vs NON using Wilcoxon rank-sum test...
  Analyzed 269 markers for EXT vs NON
Nanostring: Comparing EXT vs INT using Wilcoxon rank-sum test...
  Analyzed 269 markers for EXT vs INT
Nanostring: Comparing INT vs NON using Wilcoxon rank-sum test...
  Analyzed 269 markers for INT vs NON

Running Nanostring GSEA for EXT vs NON...
  Running GSEA for GO_Biological_Process_2021...
    Completed GO_Biological_Process_2021: Found 143 enriched pathways
  Running GSEA for GO_Molecular_Function_2021...
    Completed GO_Molecular_Function_2021: Found 23 enriched pathways
  Running GSEA for GO_Cellular_Component_2021...
    Completed GO_Cellular_Component_2021: F

2025-05-28 13:46:59,343 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!


  Analyzed 276 proteins for EXT vs INT
Olink: Comparing INT vs NON using Wilcoxon rank-sum test...
  Analyzed 276 proteins for INT vs NON

Running Olink GSEA for EXT vs NON...
  Running GSEA for GO_Biological_Process_2021...


2025-05-28 13:46:59,718 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2025-05-28 13:46:59,776 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!


    Completed GO_Biological_Process_2021: Found 326 enriched pathways
  Running GSEA for GO_Molecular_Function_2021...
    Completed GO_Molecular_Function_2021: Found 33 enriched pathways
  Running GSEA for KEGG_2021_Human...


...

    Completed GO_Biological_Process_2021: Found 327 enriched pathways
  Running GSEA for GO_Molecular_Function_2021...
    Completed GO_Molecular_Function_2021: Found 33 enriched pathways
  Running GSEA for KEGG_2021_Human...


2025-05-28 13:47:20,029 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!


    Completed KEGG_2021_Human: Found 84 enriched pathways
  Running GSEA for Reactome_2022...


2025-05-28 13:47:20,885 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!


    Completed Reactome_2022: Found 45 enriched pathways
  Running GSEA for WikiPathway_2021_Human...
    Completed WikiPathway_2021_Human: Found 56 enriched pathways
  Saved GSEA summary with 545 enriched pathways

CREATING COMPREHENSIVE VISUALIZATIONS
Checking available GSEA results...
Nanostring summaries available: ['ext_vs_non', 'ext_vs_int', 'int_vs_non']
Olink summaries available: ['ext_vs_non', 'ext_vs_int', 'int_vs_non']
Creating Nanostring summary plot...
Creating summary plot for Nanostring...
Available comparisons: ['ext_vs_non', 'ext_vs_int', 'int_vs_non']
  ext_vs_non: 387 total pathways
  ext_vs_non: 10 significant pathways (FDR < 0.25)
  ext_vs_int: 387 total pathways
  ext_vs_int: 2 significant pathways (FDR < 0.25)
  int_vs_non: 387 total pathways
  int_vs_non: 10 significant pathways (FDR < 0.25)
Combined dataframe shape: (22, 11)
  Plotting EXT VS NON: 10 pathways
  Plotting EXT VS INT: 2 pathways
  Plotting INT VS NON: 10 pathways
Saved comprehensive comparison plot